# building train data

-for a line, extract features based on this line and previous line, label=1

-for a line, extract features based on this line and a random line, label=0

-generate a large number of sents, for a low loglik generated line (bottom ranked K in the normalized-loglik list), 

-extract features for this line and a random line from the corpus, label=0

-do we need to generate a lot of sentences for the negative examples?

-we could also impose some hard coded constraints on the loglik threshold.

In [8]:
f=open('rank/test/lyrics_test_data_clean.txt','r').read().split('\n\n')
f=[i for i in f if i.strip()!='']
print len(f)

10309


In [9]:
for i in f[:200]:
    print i
    print '---------'


But don't leave your keys for me, open the door
Nobody's getting hurt here if nobody cares
Nobody's getting lost if nobody's going anywhere
---------
And darling I know that you can't be sure of anything anymore
---------
I wanna love you like it ain't no secret
Like I'm not ashamed to show
Nor would I ever, oh never never
Oh never let you go
I'll never let you go
---------
I'm on a tight rope maybe I'm scared
Maybe I love you but you never play fair
So shed a little light where nobody sees
I built a bible for the saints with my disbelief
---------
And darling I know that you can't be sure of anything anymore
---------
I wanna love you like it ain't no secret
---------
Like I'm not ashamed to show
Nor would I ever, oh never never
Oh never let you go
---------
My love is easy
Tell me bout yours and then
Tell me you need me
Carry me home again
It's been a mystery
Most of the time
Stranded on the big top
Throw me a line
---------
I wanna love you like it ain't no secret
Like I'm not asha

In [14]:
import kenlm
import gensim,os
import numpy as np
from scipy.spatial.distance import cosine
from gensim.models import Doc2Vec
import pronouncing



def is_rhyme(word1,word2):
    word1=re.sub(u'[^A-Za-z]','',word1)
    word2=re.sub(u'[^A-Za-z]','',word2)
    return int(word1 in pronouncing.rhymes(word2))
    
    
def is_rhyme_current(sent):
    sent=sent.split(' ')
    last_word=sent[-1]
    for w in sent[:-1]:
        if is_rhyme(w,last_word):
            return 1
    return 0
    
    
def get_loglik_norm(sent,LM):
    model = kenlm.LanguageModel(LM)
    loglik_norm=model.score(sent)/len(sent.split(' '))
    return loglik_norm

def get_d2v_dist(sent1,sent2,model):
    model = Doc2Vec.load(model)
    a=model.infer_vector(sent1.split(' '))
    b=model.infer_vector(sent2.split(' '))
    return cosine(a,b)



def make_feature_vec(words, model, num_features):
    # average all of the word vectors in a sentence
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def get_w2v_dist(sent1,sent2,model):
    v1=make_feature_vec(sent1,model,100)
    v2=make_feature_vec(sent2,model,100)
    return cosine(v1,v2)


def extract_features_positive(passage,LM,w2v_model,d2v_model):
    """extract feature from one passage"""
    # a passage is a consecutive set of lines without a blank line in between. we extract features with these pairs 
    # of lines as prev and next lines. they're a more coherent unit. The passages is obtained by methods above, 
    # namely, splitting the training file by '\n\n'
    line_list=passage.split('\n')
    line_list=[i for i in line_list if i!='']
    features=['loglik_norm','d2v_dist','w2v_dist','rhyme_prev','rhyme_current','len_prev','len_cur','label']
    pos_feature_vec=[]
    for i in range(1,len(line_list)):
        #extract features from the current and prev line
        prev=line_list[i-1]
        current=line_list[i]
        print('prev,current:')
        print(prev,current)
        #features
        loglik_norm=get_loglik_norm(current,LM)#LM='train3.lm'
        d2v_dist=get_d2v_dist(prev,current,d2v_model)
        w2v_dist=get_w2v_dist(prev,current,w2v_model)
        rhyme_prev=is_rhyme(prev.split(' ')[-1],current.split(' ')[-1])
        rhyme_current=is_rhyme_current(current)
        num_words_cur=len(current)
        num_words_prev=len(prev)
        label=1
        pos_feature_vec.append([loglik_norm,d2v_dist,w2v_dist,rhyme_prev,rhyme_current,num_word_prev,num_words_cur,label])
    return pos_feature_vec
        
        
        
        
        
        


ImportError: No module named kenlm

In [11]:
line_list

['',
 "But don't leave your keys for me, open the door",
 "Nobody's getting hurt here if nobody cares",
 "Nobody's getting lost if nobody's going anywhere"]

In [ ]:
d2v_model='rock_train.d2v'
w2v_model='rock_train.w2v'
LM='train3.lm'

In [ ]:
test_pas="""I wanna love you like it ain't no secret
Like I'm not ashamed to show
Nor would I ever, oh never never
Oh never let you go
I'll never let you go"""

In [18]:
a=[[-1.9321557680765789, 0.78070413044238873, 0.022806050290745117, 0, 0, 40, 28, 1], [-3.41674314226423, 0.43240190787867261, 0.23933394083644333, 0, 1, 28, 32, 1], [-1.721189308166504, 0.61283882132826939, 0.078219546226546699, 0, 0, 32, 19, 1], [-1.1092583656311035, 0.5752945981640073, 0.16501156574460696, 1, 0, 19, 21, 1]]

In [47]:
#after fixing w2v
a=[[-1.9321557680765789, 0.84771155311134316, 0.50855190776400561, 0, 0, 0, 40, 28, 1], [-3.41674314226423, 0.49395473391852707, 0.59788698690639319, 0, 0, 1, 28, 32, 1], [-1.721189308166504, 0.5148408640544081, 0.26509378411068274, 0, 0, 0, 32, 19, 1], [-1.1092583656311035, 0.25549408668095219, 0.10523526352690327, 0, 1, 0, 19, 21, 1]]

In [48]:
for i in a:print i

[-1.9321557680765789, 0.8477115531113432, 0.5085519077640056, 0, 0, 0, 40, 28, 1]
[-3.41674314226423, 0.49395473391852707, 0.5978869869063932, 0, 0, 1, 28, 32, 1]
[-1.721189308166504, 0.5148408640544081, 0.26509378411068274, 0, 0, 0, 32, 19, 1]
[-1.1092583656311035, 0.2554940866809522, 0.10523526352690327, 0, 1, 0, 19, 21, 1]


In [20]:
b=[[-2.9132102966308593, 0.84254782164524356, 0.18192692156665857, 0, 0, 25, 20, 1], [-2.3162093692355685, 0.92273771747298061, 0.13780720014199244, 0, 0, 20, 50, 1], [-2.026075839996338, 1.047541220802291, 0.13328523384166169, 0, 0, 50, 18, 1], [-1.7705413273402624, 0.97206134208362982, 0.099279340335519994, 0, 0, 18, 65, 1], [-3.4620275497436523, 1.2762789110948516, 0.034910335335743725, 0, 0, 65, 21, 1]]

In [45]:
#after fixing word2vec
b=[[-2.9132102966308593, 0.8442268296235873, 0.80551329359139057, 0, 0, 0, 25, 20, 1], [-2.3162093692355685, 0.86841709351845175, 0.86530378447712386, 0, 0, 0, 20, 50, 1], [-2.026075839996338, 0.80852565345349403, 0.70081317188750281, 0, 0, 0, 50, 18, 1], [-1.7705413273402624, 1.1240061836730386, 0.57432313199626739, 0, 0, 0, 18, 65, 1], [-3.4620275497436523, 0.87970654352248356, 0.93723039783182216, 0, 0, 0, 65, 21, 1]]

In [46]:
for i in b:print i

[-2.9132102966308593, 0.8442268296235873, 0.8055132935913906, 0, 0, 0, 25, 20, 1]
[-2.3162093692355685, 0.8684170935184518, 0.8653037844771239, 0, 0, 0, 20, 50, 1]
[-2.026075839996338, 0.808525653453494, 0.7008131718875028, 0, 0, 0, 50, 18, 1]
[-1.7705413273402624, 1.1240061836730386, 0.5743231319962674, 0, 0, 0, 18, 65, 1]
[-3.4620275497436523, 0.8797065435224836, 0.9372303978318222, 0, 0, 0, 65, 21, 1]


In [28]:
def make_feature_vec(words, model, num_features):
    # average all of the word vectors in a sentence
    words=words.split(' ')
    featureVec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    for word in words:
        nwords = nwords + 1.
        featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [42]:
from gensim.models import Word2Vec
import numpy as np
fname='rock_train.w2v'
model = Word2Vec.load(fname)
v1=make_feature_vec('I love you',model,100)
v2=make_feature_vec('I love you baby',model,100)
v3=make_feature_vec('Like I’m a broken relate',model,100)

In [39]:
pwd

u'/Users/zangsir/repo/songLyrics'

In [36]:

from scipy.spatial.distance import cosine
print cosine(v1,v2)
print cosine(v1,v3)
print cosine(v2,v3)

0.0280046085098
0.99573738118
0.988993420945


In [43]:
model["relate"]

array([ 0.01025271,  0.21873002, -0.00071767,  0.32168323, -0.42537275,
       -0.45480549,  0.03541482, -0.25030354, -0.03485415, -0.14599648,
       -0.13149096,  0.31597829, -0.08253186, -0.34015653,  0.00714215,
        0.16426595,  0.28381777,  0.26444501,  0.02581098, -0.31495187,
       -0.02961343, -0.29729041, -0.03097518,  0.07565158,  0.07872936,
       -0.02143859,  0.16349572,  0.14878665, -0.40616712,  0.04453164,
        0.41386622, -0.00784189, -0.31740722, -0.06529135, -0.44250783,
        0.15238042, -0.0681546 , -0.15394318,  0.27342334, -0.25670004,
       -0.08551664,  0.01297334, -0.68062192, -0.10403197, -0.39571747,
       -0.08757023, -0.42382604, -0.24188383, -0.01875314,  0.16733015,
        0.11461136, -0.08599471,  0.1772359 , -0.00927137, -0.00741462,
       -0.05806764,  0.23455366, -0.06418875, -0.3586829 ,  0.01317938,
        0.1891005 ,  0.28175753, -0.1886007 ,  0.26117644,  0.32812303,
       -0.20934913,  0.02118781, -0.17306197,  0.41500264,  0.16

In [3]:
import numpy as np
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6]])
np.concatenate((a, b), axis=0)


array([[1, 2],
       [3, 4],
       [5, 6]])

In [4]:
>>> xs = np.array([[1,2,3,4,5],[10,20,30,40,50]])
>>> ys = np.array([], dtype=np.int64).reshape(0,5)
>>> np.vstack([ys, xs])

array([[ 1,  2,  3,  4,  5],
       [10, 20, 30, 40, 50]])